# Working with LLM APIs

In this notebook,  we will review how to use the OpenAI API Python SDK to send prompts to the following APIs:
- OpenAI
- Google Gemini
- Anthropic 


General requirements:
- APIs keys
- Python SDKs for the APIs

<br>
<figure>
 <img src="assets/openai_sdk.png" width="100%" align="center"/></a>
<figcaption> The OpenAI API Python SDK workflow </figcaption>
</figure>

<br>

Both Google Gemini and Anthropic have their own API SDKs, which are fairly similar to the OpenAI API, and you can find examples for using those SDKs in the notebook appendix.

Let's start by import the os library:

In [1]:
from openai import OpenAI
import os

In [2]:
openai_api_key = os.environ.get("OPENAI_API_KEY")
gemini_api_key = os.environ.get("GEMINI_API_KEY")
claude_api_key = os.getenv("CLAUDE_API_KEY")

We than define the following prompt:

In [3]:
content_prompt = """
I am working with a dataset that contains information about the Red30 Tech company's product sales online.
I want to create a SQL query that calculates the total sales by state.
"""

Next, we define the number of tokens and temperature:

In [4]:
temperature = 0
max_tokens = 5000

# The OpenAI API

In this demo, we will review two models from the OpenAI API:
- [GPT4.1](https://platform.openai.com/docs/models/gpt-4.1)
- [GPT5](https://platform.openai.com/docs/models/gpt-5)

We will use the `chat.completion` method to send a GET request to the OpenAI API. The method arguments may change over time and differ between different models or be deprecated. Therefore, please check the method's [documentation](https://platform.openai.com/docs/api-reference/chat) before using it.

We will start by loading the API key and set the client:

In [5]:
client = OpenAI(api_key= openai_api_key)

> Note, the function has the OpenAI base URL as default, and therefore, we don't need to set it up.

### GPT4.1

In the first example, we will use GPT4.1 model. We will use the `message` argument to set the prompt, the `temperature` and `max_completion_tokens` arguments to define he corresponding arguments. Last but not least, we define the model as `gpt-4.1`:

In [6]:
response_gpt4_1 = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": content_prompt}],
    max_completion_tokens=max_tokens,
    temperature=temperature,
)

Let's review the output:

In [7]:
print(response_gpt4_1)

ChatCompletion(id='chatcmpl-C99DbmhJqL6JYzbmK857rmbJFRptQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Certainly! To help you write the SQL query, I'll assume your dataset has a table (let's call it `sales`) with at least the following columns:\n\n- `state` (the state where the sale occurred)\n- `sale_amount` (the amount of the sale)\n\nIf your table or column names are different, please adjust accordingly.\n\nHere’s a basic SQL query to calculate the total sales by state:\n\n```sql\nSELECT\n    state,\n    SUM(sale_amount) AS total_sales\nFROM\n    sales\nGROUP BY\n    state\nORDER BY\n    total_sales DESC;\n```\n\n**Explanation:**\n- `SUM(sale_amount)` calculates the total sales for each state.\n- `GROUP BY state` groups the results by state.\n- `ORDER BY total_sales DESC` sorts the results from highest to lowest total sales.\n\nIf your table or column names are different, let me know and I can adjust the query!", refusal=None

We will extract the response:

In [8]:
print(response_gpt4_1.choices[0].message.content)

Certainly! To help you write the SQL query, I'll assume your dataset has a table (let's call it `sales`) with at least the following columns:

- `state` (the state where the sale occurred)
- `sale_amount` (the amount of the sale)

If your table or column names are different, please adjust accordingly.

Here’s a basic SQL query to calculate the total sales by state:

```sql
SELECT
    state,
    SUM(sale_amount) AS total_sales
FROM
    sales
GROUP BY
    state
ORDER BY
    total_sales DESC;
```

**Explanation:**
- `SUM(sale_amount)` calculates the total sales for each state.
- `GROUP BY state` groups the results by state.
- `ORDER BY total_sales DESC` sorts the results from highest to lowest total sales.

If your table or column names are different, let me know and I can adjust the query!


### GPT5

Next, we will use the GTP5 model. Note that this models is a bit different from previous OpenAI models. It uses a different sampling method; therefore, the `temperature` is invalid. Instead, if you wish the model to return a deterministic response, you should use the `seed` argument.

In [9]:
response_gpt5 = client.chat.completions.create(
    model="gpt-5",
    messages=[{"role": "user", "content": content_prompt}],
    max_completion_tokens = max_tokens
)

Let's print the model response:

In [10]:
print(response_gpt5.choices[0].message.content)

Below are common SQL patterns you can use to calculate total online sales by state. Pick the one that matches your schema and adjust table/column names as needed.

1) Single sales table (state and order total stored together)
SELECT
  state,
  SUM(order_total) AS total_sales
FROM sales
WHERE channel = 'Online'
GROUP BY state
ORDER BY total_sales DESC;

2) Orders + order_items (compute sales from quantity × price)
SELECT
  o.state,
  SUM(oi.quantity * oi.unit_price) AS total_sales
FROM orders o
JOIN order_items oi
  ON oi.order_id = o.order_id
WHERE o.channel = 'Online'
GROUP BY o.state
ORDER BY total_sales DESC;

3) State stored on customers; join from orders to customers
SELECT
  c.state,
  SUM(oi.quantity * oi.unit_price) AS total_sales
FROM orders o
JOIN customers c
  ON c.customer_id = o.customer_id
JOIN order_items oi
  ON oi.order_id = o.order_id
WHERE o.channel = 'Online'
GROUP BY c.state
ORDER BY total_sales DESC;

Optional refinements:
- Date range: add AND o.order_date BETWEE

# Google Gemini API


Let's now illustrate the use the OpenAI API SDK to call the Google Gemini API. You can find more details in the [document](https://ai.google.dev/gemini-api/docs/openai).

Let's start by loading the API key, and set the client object:

In [11]:
gemini_client = OpenAI(
    api_key= gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)


Next, we will use the same workflow as we did with the GPT4.1 model:

In [12]:
response_gemini = gemini_client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=[{"role": "user", "content": content_prompt}],
    temperature=temperature,
    max_completion_tokens=max_tokens,
)


Let's print the model response:

In [13]:
print(response_gemini.choices[0].message.content)


```sql
SELECT
    State,
    SUM(Sales) AS TotalSales
FROM
    SalesData  -- Replace SalesData with the actual name of your table
GROUP BY
    State
ORDER BY
    TotalSales DESC;  -- Optional: Order the results by total sales in descending order
```

**Explanation:**

* **`SELECT State, SUM(Sales) AS TotalSales`**: This selects the `State` column and calculates the sum of the `Sales` column, aliasing it as `TotalSales`.
* **`FROM SalesData`**: This specifies the table from which to retrieve the data.  **Important:**  Replace `SalesData` with the actual name of your table in your database.
* **`GROUP BY State`**: This groups the rows based on the `State` column.  The `SUM(Sales)` function will then calculate the sum of sales for each distinct state.
* **`ORDER BY TotalSales DESC`**: This is optional. It orders the results in descending order based on the `TotalSales` column, so the state with the highest sales will appear first.  If you want ascending order (lowest sales first), remove 

# Anthropic Claude API

Likewise, we can repeat the same workflow with the Anthropic Claude API. You can find more details in this [document](https://docs.anthropic.com/en/api/openai-sdk). Let's start by setting the client:

In [14]:
claude_client = OpenAI(
    api_key=claude_api_key,
    base_url="https://api.anthropic.com/v1/",
)


Next, update the model setting and send a request to the Claude API:

In [22]:
response_claude = claude_client.chat.completions.create(
    model="claude-opus-4-1-20250805",
    messages=[{"role": "user", "content": content_prompt}],
    temperature=temperature,
    max_completion_tokens=max_tokens,
)


Last but not least, we will print the result:

In [23]:
print(response_claude.choices[0].message.content)


I'll help you create a SQL query to calculate total sales by state for Red30 Tech's online product sales. Since I don't have the exact schema of your dataset, I'll provide a few variations based on common table structures.

## Basic Query (Most Common Structure)

```sql
SELECT 
    state,
    SUM(sales_amount) AS total_sales
FROM 
    sales_table
GROUP BY 
    state
ORDER BY 
    total_sales DESC;
```

## If you have separate order and order_items tables:

```sql
SELECT 
    c.state,
    SUM(oi.quantity * oi.unit_price) AS total_sales
FROM 
    orders o
    JOIN customers c ON o.customer_id = c.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY 
    c.state
ORDER BY 
    total_sales DESC;
```

## With additional useful information:

```sql
SELECT 
    state,
    COUNT(DISTINCT order_id) AS number_of_orders,
    COUNT(DISTINCT customer_id) AS unique_customers,
    SUM(sales_amount) AS total_sales,
    ROUND(AVG(sales_amount), 2) AS average_sale
FROM 
    sales_tabl

## Appendix

In this section, we will review how to use the Google Gemini and Anthropic Python SDKs.


### Working with the Google Gemini Python SDK 

Let's start by importing the `genai` and `types` methods from the `google` library:

In [ ]:
from google import genai
from google.genai import types

Next, let's load the API key and set the client:

In [ ]:
gemini_client2 = genai.Client(api_key=gemini_api_key)

We will use the `models.generate_content` method to send the prompt to the API:

In [ ]:
response_gemini2 = gemini_client2.models.generate_content(
    model="gemini-2.0-flash",
    contents=content_prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=max_tokens, temperature=temperature
    ),
)

Let's print the output:

In [ ]:
print(response_gemini2.candidates[0].content.parts[0].text)

### Working with the Anthropic Python SDK 

Let's start by loading the `anthropic` library:

In [ ]:
import anthropic

Next, we will set the client:

In [ ]:
claude_client2 = anthropic.Anthropic(api_key=claude_api_key)

And send the prompt to the API:

In [ ]:
response_claude2 = claude_client2.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    messages=[{"role": "user", "content": content_prompt}],
)

Last but not least, we will print the response:

In [ ]:
print(response_claude2.content[0].text)

# Resources

- OpenAI API [documentation](https://platform.openai.com/docs/api-reference/introduction)
- Gemini API [documentation](https://ai.google.dev/gemini-api/docs)
- Claude API [documentation](https://docs.anthropic.com/en/home)